In [726]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import os

print(os.getcwd()) 
original_path = os.getcwd() + "/generated_graphs/1000n/WS_n1000_k13_p0.25_graph_1.gexf"
G = nx.read_gexf(original_path)
total_sources = 15
total_obv = 50
diameter = nx.diameter(G)
print(nx.diameter(G))

c:\Users\parva\OneDrive\Desktop\BTP-parva\btp_result_generation\codes\tester_structural_holes
2


In [728]:
def label_sources_and_observers(graph, n):
    total_nodes = list(graph.nodes)

    if n > len(total_nodes):
        raise ValueError("Not enough unique nodes in the graph to assign sources and observers.")

    # Randomly sample unique source and observer nodes
    source_nodes = set(random.sample(total_nodes, n))

    # Apply labels to the graph
    for node in source_nodes:
        graph.nodes[node]['label'] = 'source_nodes'


    print(f"Labeled {n} source_nodes.")
    return graph, source_nodes

In [729]:

G, sources = label_sources_and_observers(G, total_sources)
# source_nodes.clear()
# observers.clear()
# print(source_nodes)
# print(observers)



Labeled 15 source_nodes.


In [730]:
source_nodes = [n for n, d in G.nodes(data=True) if d.get('label') == 'source_nodes']


print(source_nodes)


['0', '116', '192', '288', '362', '471', '528', '567', '756', '817', '821', '828', '962', '986', '988']


Code without reserve state


In [731]:
# from collections import defaultdict
# lbl = {}
# path = {}
# my_dict = defaultdict(list)


# # Assign states to nodes: +1 for infected, -1 for susceptible
# for node in G.nodes:
#     G.nodes[node]['state'] = -1  # Susceptible
#     G.nodes[node]['prev_State'] = 0
#     G.nodes[node]['infection_time'] = -1  # Default label for uninfected
#     G.nodes[node]['obv'] = 0
#     lbl[node] = -1  # Default label -1
#     path[node] = node

# for source in source_nodes:
#     G.nodes[source]['state'] = 1
#     G.nodes[source]['infection_time'] = 0  # Infected at t=0

# # for obv in observers:
# #     G.nodes[obv]['obv'] = 1

# # Infection spread function
# def spread_infection(G, infection_rate, steps=5):
#     for t in range(1, steps + 1):
#         new_infections = []
#         for node in G.nodes:
#             if G.nodes[node]['state'] == 1:
#                 for neighbor in G.neighbors(node):
#                     if G.nodes[neighbor]['state'] == -1:
#                         if np.random.rand() < infection_rate:
#                             path[neighbor] = node
#                             new_infections.append(neighbor)
#         for ni in new_infections:
#             G.nodes[ni]['state'] = 1
#             G.nodes[ni]['infection_time'] = t  # Record time of infection

# # Simulate infection spread
# spread_infection(G, infection_rate=0.3, steps=10)



In [732]:
# # Assign labels after infection simulation
# for node in G.nodes:
#     if G.nodes[node]['infection_time'] != -1:
#         lbl[node] = G.nodes[node]['infection_time']
#     else:
#         lbl[node] = -1  # Not infected

# # Print final labels

# cnt = 0
# temp3 = 10000000
# while(temp3 > 0):
#   for node in G.nodes:
#     my_dict[node].append(G.nodes[node]['state'])
#   for node in G.nodes:
#     if(G.nodes[node]['obv'] == 0):
#       if(G.nodes[node]['infection_time'] != -1):
#         cnt = 0
#         for neighbor in G.neighbors(node):
#           if(G.nodes[neighbor]['infection_time'] != -1):
#             cnt += 1
#         for neighbor in G.neighbors(node):
#           if(G.nodes[neighbor]['infection_time'] != -1 and cnt > 0):
#             G.nodes[neighbor]['state'] += G.nodes[node]['state']/cnt
#             my_dict[neighbor].append(G.nodes[neighbor]['state'])
#         if(cnt > 0):
#           G.nodes[node]['state'] = 0
#           my_dict[neighbor].append(G.nodes[neighbor]['state'])
#     else:
#       if(G.nodes[node]['infection_time'] != -1):
#         cnt = 0
#         for neighbor in G.neighbors(node):
#           if(G.nodes[neighbor]['infection_time'] != -1 and G.nodes[neighbor]['infection_time'] < G.nodes[node]['infection_time']):
#             cnt += 1
#         for neighbor in G.neighbors(node):
#           if(G.nodes[neighbor]['infection_time'] != -1 and G.nodes[neighbor]['infection_time'] < G.nodes[node]['infection_time'] and cnt > 0):
#             G.nodes[neighbor]['state'] += G.nodes[node]['state']/cnt
#             my_dict[neighbor].append(G.nodes[neighbor]['state'])
#         if(cnt > 0):
#           G.nodes[node]['state'] = 0
#           my_dict[neighbor].append(G.nodes[neighbor]['state'])
#     temp3-=1


# for node in G.nodes:
#     print(node , " " , " node " ,G.nodes[node]['state'], " state ", G.nodes[node]['infection_time'], " ", " time ")


# for node in G.nodes:
#   average = sum(my_dict[node]) / len(my_dict[node])
#   print(node, " node " , average, " average ")

# # Optional: visualize infected vs uninfected
# plt.figure(figsize=(8, 5))
# pos = nx.spring_layout(G, seed=42)
# node_colors = ['red' if lbl[n] != -1 else 'skyblue' for n in G.nodes]
# nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=700, font_size=10)
# plt.title("Infected (red) vs Uninfected (blue) Nodes")
# plt.show()


code with reserve state


In [733]:
from collections import defaultdict
lbl = {}
path = {}
my_dict = defaultdict(list)
my_dict2 = defaultdict(list)


# Assign states to nodes: +1 for infected, -1 for susceptible
for node in G.nodes:
    G.nodes[node]['state'] = -1  # Susceptible
    G.nodes[node]['reserve_state'] = 0
    G.nodes[node]['prev_State'] = 0
    G.nodes[node]['infection_time'] = -1  # Default label for uninfected
    G.nodes[node]['obv'] = 0
    lbl[node] = -1  # Default label -1
    path[node] = node

for source in source_nodes:
    G.nodes[source]['state'] = 1
    G.nodes[source]['infection_time'] = 0  # Infected at t=0



# Infection spread function
def spread_infection(G, infection_rate, steps=5):
    for t in range(1, steps + 1):
        new_infections = []
        for node in G.nodes:
            if G.nodes[node]['state'] == 1:
                for neighbor in G.neighbors(node):
                    if G.nodes[neighbor]['state'] == -1:
                        if np.random.rand() < infection_rate:
                            path[neighbor] = node
                            new_infections.append(neighbor)
        for ni in new_infections:
            G.nodes[ni]['state'] = 1
            G.nodes[ni]['infection_time'] = t  # Record time of infection

# Simulate infection spread
spread_infection(G, infection_rate=0.4, steps=6)



In [734]:
def label_observers(graph, sources, k):
    # Get all infected nodes that are not source nodes
    infected_non_sources = [
        node for node in graph.nodes
        if graph.nodes[node]['state'] == 1 and node not in sources
    ]

    if k > len(infected_non_sources):
        raise ValueError("Not enough infected non-source nodes to assign as observers.")

    observer_nodes = set(random.sample(infected_non_sources, k))

    # Label observer nodes
    for node in observer_nodes:
        graph.nodes[node]['label'] = 'observers'
        graph.nodes[node]['obv'] = 1  # Also mark for obv flag if needed

    print(f"Labeled {k} observer nodes.")
    return observer_nodes

# Example: Select and label 3 observers
observers = label_observers(G, source_nodes, total_obv)
observer_list = list(observers)
print(observer_list)


Labeled 50 observer nodes.
['368', '38', '718', '222', '266', '327', '825', '506', '903', '94', '692', '533', '347', '15', '359', '290', '154', '393', '424', '786', '794', '773', '415', '382', '342', '63', '945', '372', '628', '313', '494', '889', '367', '876', '141', '363', '846', '501', '391', '482', '700', '884', '209', '176', '254', '548', '698', '27', '68', '559']


In [735]:
for obv in observers:
    G.nodes[obv]['obv'] = 1

In [736]:
# Assign labels after infection simulation
for node in G.nodes:
    if G.nodes[node]['infection_time'] != -1:
        lbl[node] = G.nodes[node]['infection_time']
    else:
        lbl[node] = -1  # Not infected

# Print final labels
cnt2 = 0
cnt = 0
temp3 = 10000000
flag = 1
while(temp3 > 0 and flag == 1):
  cnt2+=1
  for node in G.nodes:
    my_dict[node].append(G.nodes[node]['state'])
  flag = 0
  for node in G.nodes:
    if(G.nodes[node]['obv'] == 0):

      if(G.nodes[node]['infection_time'] != -1):
        if((G.nodes[node]['state']*(0.9))/G.degree(node) > 0.0001):
          flag = 1
          cnt = 0
          for neighbor in G.neighbors(node):
            if(G.nodes[neighbor]['infection_time'] != -1):
              cnt += 1
          for neighbor in G.neighbors(node):
            if(G.nodes[neighbor]['infection_time'] != -1 and cnt > 0):
              G.nodes[neighbor]['state'] += ((0.9)*(G.nodes[node]['state']))/cnt
              my_dict[neighbor].append(G.nodes[neighbor]['state'])
          if(cnt > 0):
            G.nodes[node]['reserve_state'] += (0.1)*G.nodes[node]['state']
            G.nodes[node]['state'] = 0
            my_dict[neighbor].append(G.nodes[neighbor]['state'])
            my_dict2[neighbor].append(G.nodes[neighbor]['reserve_state'])
    else:
      if(G.nodes[node]['infection_time'] != -1):
        if((G.nodes[node]['state']*(0.9))/G.degree(node) > 0.0001):
          flag = 1
          cnt = 0
          for neighbor in G.neighbors(node):
            if(G.nodes[neighbor]['infection_time'] != -1 and G.nodes[neighbor]['infection_time'] < G.nodes[node]['infection_time']):
              cnt += 1
          for neighbor in G.neighbors(node):
            if(G.nodes[neighbor]['infection_time'] != -1 and G.nodes[neighbor]['infection_time'] < G.nodes[node]['infection_time'] and cnt > 0):
              G.nodes[neighbor]['state'] += ((0.9)*(G.nodes[node]['state']))/cnt
              my_dict[neighbor].append(G.nodes[neighbor]['state'])
          if(cnt > 0):
            G.nodes[node]['reserve_state'] += (0.1)*G.nodes[node]['state']
            G.nodes[node]['state'] = 0
            my_dict[neighbor].append(G.nodes[neighbor]['state'])
            my_dict2[neighbor].append(G.nodes[neighbor]['reserve_state'])
    temp3-=1


for node in G.nodes:
    print(node , " " , " node " ,G.nodes[node]['state'], " state ", G.nodes[node]['infection_time'], " ", " time ")

print("state value")
for node in G.nodes:
  average = sum(my_dict[node]) / len(my_dict[node])
  print(node, " node " , average, " average ")

print("reserved state value")
for node in G.nodes:
  print(node , " " , " node " ,G.nodes[node]['reserve_state'], " reserve_state ")


print(source_nodes)
print(observers)
print("total itrs are: ", cnt2)
# # Optional: visualize infected vs uninfected
# plt.figure(figsize=(8, 5))
# pos = nx.spring_layout(G, seed=42)
# node_colors = ['red' if lbl[n] != -1 else 'skyblue' for n in G.nodes]
# nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=700, font_size=10)
# plt.title("Infected (red) vs Uninfected (blue) Nodes")
# plt.show()

0    node  0.0021324658188507056  state  0    time 
1    node  0.006988068639401108  state  1    time 
2    node  0.008582557579476038  state  1    time 
3    node  0.007588454418809547  state  2    time 
4    node  0.0075430191862554345  state  1    time 
5    node  0.006632778761409262  state  2    time 
6    node  0.005877514175720233  state  1    time 
7    node  0.0071851314167102895  state  1    time 
8    node  0.006584474729369923  state  1    time 
9    node  0.00764839176829622  state  1    time 
10    node  0.008477219420120734  state  1    time 
11    node  0.00546840136447219  state  2    time 
12    node  0.00667372503578381  state  2    time 
13    node  0.007636941068762373  state  1    time 
14    node  0.005213056888331419  state  2    time 
15    node  0.007442995386499367  state  2    time 
16    node  0.007748463278422981  state  1    time 
17    node  0.008717290900909109  state  1    time 
18    node  0.007668862539015199  state  1    time 
19    node  0.00734500

In [737]:
# import matplotlib.pyplot as plt
# import networkx as nx
# from matplotlib.patches import Patch

# # Visualization: highlight source_nodes and observers
# plt.figure(figsize=(10, 6))
# pos = nx.spring_layout(G, seed=42)

# # Determine node colors
# node_colors = []
# for node in G.nodes:
#     if node in source_nodes:
#         node_colors.append('green')       # Source nodes
#     elif node in observers:
#         node_colors.append('orange')      # Observer nodes
#     else:
#         node_colors.append('lightgray')   # All other nodes

# # Draw the network
# nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='gray',
#         node_size=700, font_size=10)

# # Legend
# legend_elements = [
#     Patch(facecolor='green', edgecolor='black', label='Source Nodes'),
#     Patch(facecolor='orange', edgecolor='black', label='Observer Nodes'),
#     Patch(facecolor='lightgray', edgecolor='black', label='Other Nodes')
# ]
# plt.legend(handles=legend_elements, loc='best')
# plt.title("Network Visualization: Source and Observer Nodes")
# plt.show()


reserve state based source

In [738]:
for obv in observers:
    G.nodes[obv]['obv'] = 1

In [739]:
# Define the number of potential sources you want to label
n = total_sources  # You can change this value as needed

# Find candidates that are not observers
non_observer_nodes = [node for node in G.nodes if G.nodes[node]['obv'] == 0]

# Sort the non-observer nodes based on reserve_state in descending order
sorted_candidates = sorted(non_observer_nodes, key=lambda x: G.nodes[x]['reserve_state'], reverse=True)

# Create and store top-n potential sources
Reserve_state_based_source = sorted_candidates[:n]

# Label top n nodes as potential_source
for i in range(min(n, len(sorted_candidates))):
    G.nodes[sorted_candidates[i]]['potential_source'] = 1
else:
    for i in range(n, len(sorted_candidates)):
        G.nodes[sorted_candidates[i]]['potential_source'] = 0

# Print out the selected potential sources
print(f"\nTop {n} potential sources based on reserve_state:")
for node in Reserve_state_based_source:
    print(f"Node {node} - Reserve State: {G.nodes[node]['reserve_state']}")

# Also print the list directly
print("\nReserve_state_based_source list:")
print(Reserve_state_based_source)

# # Optional: Visualize potential sources
# plt.figure(figsize=(8, 5))
# node_colors = []
# for node in G.nodes:
#     if G.nodes[node].get('potential_source', 0) == 1:
#         node_colors.append('green')  # Potential sources in green
#     elif lbl[node] != -1:
#         node_colors.append('red')    # Infected nodes in red
#     else:
#         node_colors.append('skyblue')  # Others in blue

# nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=700, font_size=10)
# plt.title("Infected (red), Uninfected (blue), Potential Sources (green)")
# plt.show()



Top 15 potential sources based on reserve_state:
Node 471 - Reserve State: 4.023559262622559
Node 528 - Reserve State: 3.938762430759025
Node 192 - Reserve State: 3.677739711124527
Node 116 - Reserve State: 3.6681267618688795
Node 828 - Reserve State: 3.6213052966191035
Node 288 - Reserve State: 3.3275348333695005
Node 821 - Reserve State: 3.2835730964298464
Node 988 - Reserve State: 2.4815133011761676
Node 0 - Reserve State: 2.216964399187973
Node 756 - Reserve State: 1.9768213432856558
Node 986 - Reserve State: 1.917523905078299
Node 962 - Reserve State: 1.8122606612674972
Node 817 - Reserve State: 1.7163668770084284
Node 362 - Reserve State: 1.6853338144268868
Node 567 - Reserve State: 1.6675723530501623

Reserve_state_based_source list:
['471', '528', '192', '116', '828', '288', '821', '988', '0', '756', '986', '962', '817', '362', '567']


State based source


In [740]:
import networkx as nx
import matplotlib.pyplot as plt

# Ensure all node IDs in the graph are strings
G = nx.relabel_nodes(G, lambda x: str(x))

# Number of state-based sources to label
n = total_sources  # You can change this value

# Find candidates that are not observers
non_observer_nodes = [node for node in G.nodes if G.nodes[node].get('obv', 0) == 0]

# Sort the non-observer nodes by their 'state' attribute in descending order
sorted_candidates = sorted(non_observer_nodes, key=lambda x: G.nodes[x]['state'], reverse=True)

# Select top-n candidates as state-based sources
State_based_source = sorted_candidates[:n]

# Label them in the graph
for i, node in enumerate(sorted_candidates):
    G.nodes[node]['state_based_source'] = 1 if i < n else 0

# Print selected sources
print(f"\nTop {n} state-based sources based on node state:")
for node in State_based_source:
    print(f"Node {node} - State: {G.nodes[node]['state']}")

print("\nState_based_source list:")
print(State_based_source)

# # Positioning for visualization
# pos = nx.spring_layout(G, seed=42)  # Use consistent layout

# # Assign colors based on node roles
# node_colors = []
# for node in G.nodes:
#     if G.nodes[node].get('state_based_source', 0) == 1:
#         node_colors.append('orange')       # State-based sources
#     elif lbl[node] != -1:
#         node_colors.append('crimson')      # Infected
#     else:
#         node_colors.append('lightgray')    # Others

# # Draw graph
# plt.figure(figsize=(8, 5))
# nx.draw(
#     G, pos, with_labels=True,
#     node_color=node_colors,
#     node_size=700,
#     font_size=10,
#     font_color='black',
#     edge_color='gray'
# )

# plt.title("Infected (crimson), Uninfected (gray), State-Based Sources (orange)", fontsize=12)
# plt.tight_layout()
# plt.show()



Top 15 state-based sources based on node state:
Node 478 - State: 0.018311404915907766
Node 523 - State: 0.018275712021801543
Node 507 - State: 0.018049584457044864
Node 454 - State: 0.018014968842428015
Node 462 - State: 0.017929627124387858
Node 455 - State: 0.017897651753447968
Node 519 - State: 0.017815261158221338
Node 475 - State: 0.017738706870773148
Node 477 - State: 0.017612015335723333
Node 456 - State: 0.017602927700827883
Node 452 - State: 0.01755770314548178
Node 540 - State: 0.01748295347315068
Node 544 - State: 0.01748265823208731
Node 539 - State: 0.01741711460652463
Node 492 - State: 0.017333180704903015

State_based_source list:
['478', '523', '507', '454', '462', '455', '519', '475', '477', '456', '452', '540', '544', '539', '492']


In [741]:
# import matplotlib.pyplot as plt
# import networkx as nx
# from matplotlib.patches import Patch

# # Create subplots in a 2x2 layout
# fig, axs = plt.subplots(2, 2, figsize=(18, 12))
# fig.suptitle("Network Analysis Summary", fontsize=16)

# # Use the same layout for all subplots
# pos = nx.spring_layout(G, seed=42)

# # --------- Top Left: Infected vs Uninfected ----------
# node_colors_1 = ['red' if lbl[n] != -1 else 'skyblue' for n in G.nodes]
# nx.draw(G, pos, with_labels=True, node_color=node_colors_1, node_size=700, font_size=10, ax=axs[0, 0])
# axs[0, 0].set_title("Infected (red) vs Uninfected (blue) Nodes")

# # --------- Top Right: Source and Observer Nodes ----------
# node_colors_2 = []
# for node in G.nodes:
#     if node in source_nodes:
#         node_colors_2.append('green')       # Source nodes
#     elif node in observers:
#         node_colors_2.append('orange')      # Observer nodes
#     else:
#         node_colors_2.append('lightgray')   # All other nodes
# nx.draw(G, pos, with_labels=True, node_color=node_colors_2, edge_color='gray',
#         node_size=700, font_size=10, ax=axs[0, 1])
# axs[0, 1].set_title("Source Nodes (green) & Observer Nodes (orange)")

# # --------- Bottom Left: Reserve State Based Sources ----------
# node_colors_3 = []
# for node in G.nodes:
#     if G.nodes[node].get('potential_source', 0) == 1:
#         node_colors_3.append('green')  # Potential sources in green
#     elif lbl[node] != -1:
#         node_colors_3.append('red')    # Infected nodes in red
#     else:
#         node_colors_3.append('skyblue')  # Others in blue
# nx.draw(G, pos, with_labels=True, node_color=node_colors_3, node_size=700, font_size=10, ax=axs[1, 0])
# axs[1, 0].set_title("Potential Sources (green) Based on Reserve State")

# # --------- Bottom Right: State Based Sources ----------
# node_colors_4 = []
# for node in G.nodes:
#     if G.nodes[node].get('state_based_source', 0) == 1:
#         node_colors_4.append('orange')      # Potential sources in orange
#     elif lbl[node] != -1:
#         node_colors_4.append('crimson')     # Infected nodes in crimson
#     else:
#         node_colors_4.append('lightgray')   # Others in light gray
# nx.draw(G, pos, with_labels=True, node_color=node_colors_4, node_size=700, font_size=10, ax=axs[1, 1])
# axs[1, 1].set_title("Potential Sources (orange) Based on State")

# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.show()


In [742]:
# Ensure both lists have same type (strings)
source_nodes = [str(node) for node in source_nodes]
Reserve_state_based_source = [str(node) for node in Reserve_state_based_source]

# 1. Find common and uncommon elements
common_sources_reservebased = list(set(source_nodes) & set(Reserve_state_based_source))
uncommon_source_nodes = list(set(source_nodes) - set(common_sources_reservebased))
uncommon_reserve_sources = list(set(Reserve_state_based_source) - set(common_sources_reservebased))

# 2. Print the lists
print("Common sources:", common_sources_reservebased)
print("Uncommon source_nodes:", uncommon_source_nodes)
print("Uncommon Reserve_state_based_source:", uncommon_reserve_sources)

# 3. Path calculations (between original source_nodes and Reserve_state_based_source)
print("\nPath Calculations:")
for i in uncommon_source_nodes:
    for j in uncommon_reserve_sources:
        print("\nsource:", i, "target:", j)
        try:
            path = nx.shortest_path(G, source=i, target=j)
            print("Path:", path)
            path_length = len(path) - 1
        except nx.NetworkXNoPath:
            print(f"No path between {i} and {j}")
            path_length = float('inf')

        print("Path length:", path_length)


Common sources: ['0', '756', '362', '962', '288', '821', '528', '116', '986', '567', '988', '828', '817', '192', '471']
Uncommon source_nodes: []
Uncommon Reserve_state_based_source: []

Path Calculations:


In [743]:
import matplotlib.pyplot as plt
import networkx as nx

B = nx.Graph()
B.add_nodes_from(uncommon_source_nodes, bipartite=0)
B.add_nodes_from(uncommon_reserve_sources, bipartite=1)

edge_colors = []
edge_labels = {}

for i in uncommon_source_nodes:
    for j in uncommon_reserve_sources:
        try:
            wt = len(nx.shortest_path(G, source=str(i), target=str(j))) - 1
            edge_colors.append('black')
        except nx.NetworkXNoPath:
            print(f"No path between {i} and {j}, setting weight to 'inf'")
            wt = float('inf')
            edge_colors.append('red')
        B.add_edge(str(i), str(j), weight=wt)
        edge_labels[(str(i), str(j))] = wt





In [744]:
matching = nx.algorithms.matching.min_weight_matching(B, weight='weight')

# Extract actual weights of the matching
total_weight = 0
for u, v in matching:
    # Ensure consistent node order (optional)
    weight = B[u][v]['weight']
    print(f"Matched: {u} - {v}, weight = {weight}")
    total_weight += weight

print(f"Total matching weight: {total_weight}")
print(f"average matching weight: {total_weight/len(source_nodes)}")

Total matching weight: 0
average matching weight: 0.0


In [745]:
# source_nodes
# State_based_source

# for i in source_nodes:
#     for j in State_based_source:
#         print("source", i, j)
#         try:
#             path = nx.shortest_path(G, source=i, target=j)
#             print(path)
#             path_length = len(path) - 1
#         except nx.NetworkXNoPath:
#             print(f"No path between {i} and {j}")
#             path_length = float('inf')  # or use 'inf' if you want a string

#         print(path_length)


In [746]:
# B = nx.Graph()
# B.add_nodes_from(source_nodes, bipartite=0)
# B.add_nodes_from(State_based_source, bipartite=1)

# edge_colors = []
# edge_labels = {}

# for i in source_nodes:
#     for j in State_based_source:
#         try:
#             wt = len(nx.shortest_path(G, source=str(i), target=str(j))) - 1
#             edge_colors.append('black')
#         except nx.NetworkXNoPath:
#             print(f"No path between {i} and {j}, setting weight to 'inf'")
#             wt = float('inf')
#             edge_colors.append('red')
#         B.add_edge(str(i), str(j), weight=wt)
#         edge_labels[(str(i), str(j))] = wt





In [747]:
# matching = nx.algorithms.matching.min_weight_matching(B, weight='weight')

# # Extract actual weights of the matching
# total_weight = 0
# for u, v in matching:
#     # Ensure consistent node order (optional)
#     weight = B[u][v]['weight']
#     print(f"Matched: {u} - {v}, weight = {weight}")
#     total_weight += weight

# print(f"Total matching weight: {total_weight}")
# print(f"average matching weight: {total_weight/2}")

In [748]:
import networkx as nx
import matplotlib.pyplot as plt

# Step 0: Ensure consistent types (strings)
source_nodes = [str(node) for node in source_nodes]
State_based_source = [str(node) for node in State_based_source]

# Step 1: Find common and uncommon elements
common_sources_statebased = list(set(source_nodes) & set(State_based_source))
uncommon_source_nodes = list(set(source_nodes) - set(common_sources_statebased))
uncommon_state_sources = list(set(State_based_source) - set(common_sources_statebased))

print(" Common sources:", common_sources_statebased)
print(" Uncommon source_nodes:", uncommon_source_nodes)
print(" Uncommon State_based_source:", uncommon_state_sources)

# Step 2: Shortest path analysis between all source and state source nodes
print("\n Path Calculations:")
for i in source_nodes:
    for j in State_based_source:
        print("\nsource:", i, "target:", j)
        try:
            path = nx.shortest_path(G, source=i, target=j)
            print("Path:", path)
            path_length = len(path) - 1
        except nx.NetworkXNoPath:
            print(f"No path between {i} and {j}")
            path_length = float('inf')

        print("Path length:", path_length)

# Step 3: Build bipartite graph
B = nx.Graph()
B.add_nodes_from(source_nodes, bipartite=0)
B.add_nodes_from(State_based_source, bipartite=1)

edge_colors = []
edge_labels = {}

for i in source_nodes:
    for j in State_based_source:
        try:
            wt = len(nx.shortest_path(G, source=str(i), target=str(j))) - 1
            edge_colors.append('black')
        except nx.NetworkXNoPath:
            print(f"No path between {i} and {j}, setting weight to 'inf'")
            wt = float('inf')
            edge_colors.append('red')
        B.add_edge(str(i), str(j), weight=wt)
        edge_labels[(str(i), str(j))] = wt

# Step 4: Compute minimum weight matching
matching = nx.algorithms.matching.min_weight_matching(B, weight='weight')

# Step 5: Extract weights
total_weight_state = 0
print("\n Matching:")
for u, v in matching:
    weight = B[u][v]['weight']
    print(f"Matched: {u} - {v}, weight = {weight}")
    total_weight_state += weight

print(f"\nTotal matching weight: {total_weight_state}")
print(f"Average matching weight: { total_weight_state /len(source_nodes)}")


 Common sources: []
 Uncommon source_nodes: ['0', '756', '362', '962', '288', '821', '528', '116', '567', '986', '988', '828', '817', '192', '471']
 Uncommon State_based_source: ['454', '523', '540', '478', '456', '455', '544', '492', '507', '477', '539', '519', '475', '452', '462']

 Path Calculations:

source: 0 target: 478
Path: ['0', '103', '478']
Path length: 2

source: 0 target: 523
Path: ['0', '523']
Path length: 1

source: 0 target: 507
Path: ['0', '10', '507']
Path length: 2

source: 0 target: 454
Path: ['0', '22', '454']
Path length: 2

source: 0 target: 462
Path: ['0', '462']
Path length: 1

source: 0 target: 455
Path: ['0', '22', '455']
Path length: 2

source: 0 target: 519
Path: ['0', '36', '519']
Path length: 2

source: 0 target: 475
Path: ['0', '8', '475']
Path length: 2

source: 0 target: 477
Path: ['0', '58', '477']
Path length: 2

source: 0 target: 456
Path: ['0', '25', '456']
Path length: 2

source: 0 target: 452
Path: ['0', '20', '452']
Path length: 2

source: 0 tar

In [749]:
import os
import pandas as pd

# Original path
original_path 

# Get directory and filename without extension
base_dir = os.path.dirname(original_path)
base_name = os.path.splitext(os.path.basename(original_path))[0]
base_dir = os.getcwd() + "/results"
print(base_dir)
print(base_name)
# Create new filename for the final GEXF
final_gexf_path = os.path.join(base_dir, base_name  + "_" +str(len(source_nodes)) + "structural_holes_final.gexf")

# Save the graph
nx.write_gexf(G, final_gexf_path)
print(f"Graph saved to: {final_gexf_path}")

# Prepare CSV export
data = []
for node, attrs in G.nodes(data=True):
    row = {'node': node}
    row.update(attrs)
    data.append(row)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
csv_path = os.path.join(base_dir, base_name + "_" +str(len(source_nodes)) +"structural_holes_final_attributes.csv")
graph_name = base_name
df.to_csv(csv_path, index=False)
print(f"Node attributes saved to: {csv_path}")


c:\Users\parva\OneDrive\Desktop\BTP-parva\btp_result_generation\codes\tester_structural_holes/results
ER_n1000_p0.15_graph_10
Graph saved to: c:\Users\parva\OneDrive\Desktop\BTP-parva\btp_result_generation\codes\tester_structural_holes/results\ER_n1000_p0.15_graph_10_15structural_holes_final.gexf
Node attributes saved to: c:\Users\parva\OneDrive\Desktop\BTP-parva\btp_result_generation\codes\tester_structural_holes/results\ER_n1000_p0.15_graph_10_15structural_holes_final_attributes.csv


In [750]:
import os
import csv

# Assuming these variables already exist from your previous code:
# source_nodes: List of source nodes
# identified_reserve_nodes, avg_weight_reserve, common_reserve: Precomputed for reserve state
# identified_state_nodes, avg_weight_state, common_state: Precomputed for state
# graph_name: The name of the graph

# --- Prepare the row ---
row = [
    graph_name,
    len(source_nodes),
    source_nodes,  # Number of sources
    diameter,
    len(Reserve_state_based_source),
    Reserve_state_based_source,  # Identified (Reserve)
    round(total_weight/len(source_nodes), 3),  # Avg Weight (Reserve)
    len(common_sources_reservebased),
    common_sources_reservebased,  # Common (Reserve)
    len(State_based_source),
    State_based_source,  # Identified (State)
    round(total_weight_state/len(source_nodes), 3),  # Avg Weight (State)
    len(common_sources_statebased),
    common_sources_statebased,  # Common (State)
    observer_list
]

# --- Write results to CSV ---
csv_filename = "results_compilation_structural_holes.csv"
write_header = not os.path.exists(csv_filename)

with open(csv_filename, mode='a', newline='') as file:
    writer = csv.writer(file)
    if write_header:
         writer.writerow([
            "Graph Name",
            "no of sources",
            "#Sources",
            "diameter",
            "no of identefied sources_Reserve(diff from original)",
            "#Identified (Reserve)",
            "Avg Weight (Reserve)",
            "len common reserve",
            "#Common (Reserve)",
            "no of identefied sources_State(diff from original) ",
            "#Identified (State)",
            "Avg Weight (State)",
            "len common state",
            "#Common (State)",
            "observer list"
        ])  
    writer.writerow(row)

print(f"[✔] Results appended to {csv_filename}")


[✔] Results appended to results_compilation_structural_holes.csv
